## Use this starter notebook to run some new experiments

In [1]:
# Change path as needed
MAP_PATH = "../tests/pickle_data/GresvanAIE.xz"

In [2]:
# load cell magic things
%load_ext line_profiler
%load_ext Cython

In [3]:
# imports
import matplotlib.pyplot as plt
import numpy as np

from sc2.ids.unit_typeid import UnitTypeId
from sc2.bot_ai import BotAI
from sc2.position import Point2
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.game_data import Race
from sc2.unit import Unit
from sc2.units import Units

from tests.load_bot_from_pickle import get_map_specific_bot

In [4]:
# setup a burnysc2 BOTAI instance we can test with
bot: BotAI = get_map_specific_bot(MAP_PATH)

In [6]:
from sc2.constants import CREATION_ABILITY_FIX
from sc2.ids.ability_id import AbilityId
from sc2.ids.unit_typeid import UnitTypeId
from collections import Counter
from sc2.data import Race
from sc2.unit import Unit

def _abilities_count_and_build_progress(self):
    """Cache for the already_pending function, includes protoss units warping in,
    all units in production and all structures, and all morphs"""
    abilities_amount: Counter[AbilityId] = Counter()
    unit: Unit
    for unit in self.units + self.structures:
        for order in unit.orders:
            abilities_amount[order.ability.exact_id] += 1
        if not unit.is_ready and (self.race != Race.Terran or not unit.is_structure):
            # If an SCV is constructing a building, already_pending would count this structure twice
            # (once from the SCV order, and once from "not structure.is_ready")
            if unit.type_id in CREATION_ABILITY_FIX:
                if unit.type_id == UnitTypeId.ARCHON:
                    # Hotfix for archons in morph state
                    creation_ability = AbilityId.ARCHON_WARP_TARGET
                    abilities_amount[creation_ability] += 2
                else:
                    # Hotfix for rich geysirs
                    creation_ability = CREATION_ABILITY_FIX[unit.type_id]
                    abilities_amount[creation_ability] += 1
            else:
                creation_ability: AbilityId = self.game_data.units[unit.type_id.value].creation_ability.exact_id
                abilities_amount[creation_ability] += 1

    return abilities_amount

In [22]:
%timeit _abilities_count_and_build_progress(bot)

19.2 μs ± 447 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Proposed update to `abilities_count_structures`

In [92]:
%%cython -a

# cython: boundscheck=False, wraparound=False, cdivision=True
"""
Optimized Cython implementation for tracking ability counts and build progress.
Replaces the Python _abilities_count_and_build_progress method for maximum speed.
"""

from cython_extensions.ability_mapping import map_value

from sc2.data import Race
from sc2.ids.ability_id import AbilityId
from cython cimport boundscheck, wraparound
from libc.string cimport memset
from cpython.mem cimport PyMem_Malloc, PyMem_Free

cdef int STRUCT_ABILITIES[1620]  # raw C array

memset(STRUCT_ABILITIES, 0, 1620 * sizeof(int))
STRUCT_ABILITIES[AbilityId.BUILD_REACTOR_STARPORT.value] = 1
STRUCT_ABILITIES[AbilityId.BUILD_TECHLAB_STARPORT.value] = 1
STRUCT_ABILITIES[AbilityId.TERRANBUILD_COMMANDCENTER.value] = 2
STRUCT_ABILITIES[AbilityId.ZERGBUILD_HATCHERY.value] = 2
STRUCT_ABILITIES[AbilityId.UPGRADETOLAIR_LAIR.value] = 2
STRUCT_ABILITIES[AbilityId.UPGRADETOHIVE_HIVE.value] = 1

cdef struct AbilityCount:
    int ability_id
    unsigned int count

cdef class AbilityBuffer:
    cdef AbilityCount* ptr
    cdef int size

    def __cinit__(self, int size):
        self.size = size
        self.ptr = <AbilityCount*> PyMem_Malloc(size * sizeof(AbilityCount))
        memset(self.ptr, 0, size * sizeof(AbilityCount))

    def __dealloc__(self):
        if self.ptr != NULL:
            PyMem_Free(self.ptr)
            self.ptr = NULL

    property mv:
        def __get__(self):
            # Return a Python memoryview
            return <AbilityCount[:self.size]> self.ptr


# Disable Python checks for speed
@boundscheck(False)
@wraparound(False)
cpdef AbilityCount[:] abilities_count_structures(object bot):
    cdef:
        unsigned int MIN_ABILITIES = 0
        unsigned int MAX_ABILITIES = 2200
        AbilityBuffer buf = AbilityBuffer(MAX_ABILITIES)
        AbilityCount* arr = buf.ptr

        object unit, order, orders
        int ability_id
        # special ability ids that should always be counted if seen

        object structures = bot.structures
        object workers = bot.workers
        bint is_protoss = bot.race == Race.Protoss

        unsigned int len_structures = len(structures)
        unsigned int len_workers = len(workers)
        unsigned int len_orders, i, j, struct_ability_int, unit_id_int

        double completed_build_progress = 1.0


    # Workers loop
    for i in range(len_workers):
        unit = workers[i]
        orders = unit.orders
        len_orders = len(orders)
        for j in range(len_orders):
            # cache proto on stack (faster)
            order = orders[j]
            ability_id = <int> order.ability._proto.ability_id
            if MIN_ABILITIES <= ability_id < MAX_ABILITIES:
                arr[ability_id].count += 1

    # Structures
    if is_protoss:
        for i in range(len_structures):
            unit = structures[i]
            if <double> unit.build_progress < completed_build_progress:
                unit_id_int = unit._proto.unit_type
                ability_id = <int> map_value(unit_id_int)
                if ability_id != -1:
                    arr[ability_id].count += 1
    else:
        for i in range(len_structures):
            unit = structures[i]
            unit_id_int = unit._proto.unit_type
            ability_id = <int> map_value(unit_id_int)
            if <double> unit.build_progress < completed_build_progress:
                if STRUCT_ABILITIES[ability_id]:
                    arr[ability_id].count += 1
            elif STRUCT_ABILITIES[ability_id] == 2:
                orders = unit.orders
                len_orders = len(orders)
                for j in range(len_orders):
                    order = orders[j]
                    ability_id = <int> order.ability._proto.ability_id
                    arr[ability_id].count += 1

    return buf.mv

@boundscheck(False)
@wraparound(False)
cpdef unsigned int cy_structure_pending(
        object bot,
        object structure_type,
    ):
    cdef:
        unsigned int num_pending = 0
        object counts_and_progress
        int target = <int> structure_type.value
        AbilityCount item
        Py_ssize_t arr_len
        int target_created_ability



    # Use optimized Cython function to get ability counts

    counts_and_progress = abilities_count_structures(bot) #returns a c array memoryview

    arr_len = counts_and_progress.shape[0]
    target_created_ability = <int> map_value(target)

    if 0 <= target_created_ability < arr_len:
        item = counts_and_progress[target_created_ability]
        num_pending += item.count
    return num_pending

Content of stderr:
/home/tom/.cache/ipython/cython/_cython_magic_d5980eed60e747e2ab74b90ad727230334c92ac8ce174e5c49bf163aa96b7275.c: In function ‘__pyx_f_78_cython_magic_d5980eed60e747e2ab74b90ad727230334c92ac8ce174e5c49bf163aa96b7275_abilities_count_structures’:
/home/tom/.cache/ipython/cython/_cython_magic_d5980eed60e747e2ab74b90ad727230334c92ac8ce174e5c49bf163aa96b7275.c:18274:42: warning: comparison of integer expressions of different signedness: ‘unsigned int’ and ‘int’ [-Wsign-compare]
18274 |       __pyx_t_7 = (__pyx_v_MIN_ABILITIES <= __pyx_v_ability_id);
      |                                          ^~
/home/tom/.cache/ipython/cython/_cython_magic_d5980eed60e747e2ab74b90ad727230334c92ac8ce174e5c49bf163aa96b7275.c:18276:41: warning: comparison of integer expressions of different signedness: ‘int’ and ‘unsigned int’ [-Wsign-compare]
18276 |         __pyx_t_7 = (__pyx_v_ability_id < __pyx_v_MAX_ABILITIES);
      |                                         ^

In [93]:
%timeit abilities_count_structures(bot)

4.17 μs ± 58.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [88]:
%timeit cy_structure_pending(bot, UnitTypeId.BARRACKS)

5.12 μs ± 322 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [89]:
%timeit bot.already_pending(UnitTypeId.BARRACKS)

1.74 μs ± 4.97 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [77]:
worker=bot.workers.first

In [25]:
%timeit worker.orders[0].ability.exact_id

525 ns ± 47.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [26]:
%timeit worker.orders[0].ability._proto.ability_id

127 ns ± 1.73 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [34]:
worker.orders[0].ability.exact_id.value

295

In [35]:
worker.orders[0].ability._proto.ability_id

295

In [79]:
%timeit worker.type_id.value

425 ns ± 7.77 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [81]:
%timeit worker._proto.unit_type

90.9 ns ± 0.235 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [5]:
structure = bot.structures.first

In [6]:
%timeit structure.build_progress

143 ns ± 5.52 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [7]:
%timeit structure._proto.build_progress

117 ns ± 2.23 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
